# Information Retrieval Project (Phase 2)

## Import Libraries

In [1]:
import os
import sys
import json
import pickle
import re
import math
from string import punctuation
from collections import defaultdict
from parsivar import Normalizer, Tokenizer, FindStems

## Load Dataset

### Structure of dataset
- dataset 
    - docID
        - title
        - content
        - tags
        - date
        - url
        - category

In [15]:
docs_path = './IR_data_news_12k.json'

def load_docs(docs_path):
    result = {}
    with open(docs_path) as f:
        docs = json.load(f)
        for docID, body in docs.items():
            result[docID] = {}
            result[docID]['title'] = body['title']
            result[docID]['content'] = body['content']
            result[docID]['url'] = body['url']
            result[docID]['length'] = 0
    return result

docs = load_docs(docs_path)

## Preprocess Data

### Control Preprocessing Constants

In [2]:
# whether remove stopwords or not
stopwords_remove=True

# whether stem the terms or not
stemming=True

### Remove Stopwords

In [3]:
def load_stopwords(stopwords_path):
    stopwords_set = None
    with open(stopwords_path, 'r') as f:
        stopwords_set = set(f.read().split('\n'))
    return stopwords_set

In [4]:
def remove_stopwords(words, persian_stopwords_path='./persian-stopwords.txt'):
    persian_stopwords = load_stopwords(persian_stopwords_path)
    return [word for word in words if word not in persian_stopwords]

### Stacking Preprocessing components to make a pipeline

In [5]:
def preprocess(text, stopwords_remove=True, stemming=True):
    normalizer = Normalizer()
    tokenizer = Tokenizer()
    stemmer = FindStems()
    
    pure_text = re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)
    normal_text = normalizer.normalize(pure_text)
    res = tokenizer.tokenize_words(normal_text)
    if stemming:
        res = list(map(stemmer.convert_to_stem, res))
    if stopwords_remove:
        res = remove_stopwords(res)
    
    return res

print(preprocess('سلام من امروز می خواهم این تابع را ۳ بار امتحان کنم.', stopwords_remove=stopwords_remove, stemming=stemming))

['سلام', 'امروز', 'خواست&خواه', 'تابع', '3', 'امتحان', 'کرد&کن']


In [16]:
def preprocess_contents(docs_dict, stopwords_remove=True, stemming=True):
    normalizer = Normalizer()
    tokenizer = Tokenizer()
    stemmer = FindStems()
    
    for docID, body in docs_dict.items():
        body['content'] = preprocess(body['content'], stopwords_remove, stemming)
        
    return docs_dict

preprocessed_docs = preprocess_contents(load_docs(docs_path), stopwords_remove=stopwords_remove, stemming=stemming)

## Score Functions

### TFIDF Score

$$tfidf(t, d, D) = tf(t, d) \times idf(t, D) = \log(1 + f_{t, d}) \times \log(\frac{N}{n_t})$$

In [18]:
def compute_tfidf(term_frequency, document_frequency, collection_size):
    return math.log(term_frequency + 1) * math.log(collection_size / document_frequency)

In [19]:
def tfidf(term, document, collection, do_preprocess=False):
    if do_preprocess:
        term = preprocess(term, stopwords_remove=stopwords_remove, stemming=stemming)[0]
    # compute term frequency
    term_frequency = document['content'].count(term)
    # compute document frequency
    document_frequency = sum([1 for _, doc in collection.items() if term in doc['content']])
    collection_size = len(collection)
    
    # compute final tdidf 
    return compute_tfidf(term_frequency, document_frequency, collection_size)

In [10]:
tfidf('خبرگزاری', preprocessed_docs['3'], preprocessed_docs)

0.011570860416470444

## Positional Index

Structure of Index:
- Token
    - Total Appearance
    - Champion List
    - docs
        - list of docs
            - total incidences in every doc
            - list of each position in every doc
            - TFIDF

In [20]:
def save_index(index, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(index, outp, pickle.HIGHEST_PROTOCOL)
        print(f'index saved in {filename}')
    
def load_index(filename):
    index = None
    with open(filename, 'rb') as inp:
        index = pickle.load(inp)
    return index

In [21]:
index = load_index('index_v2.pkl')
dictionary = load_index('dictionary_v2.pkl')

## Update Index
Add `tfidf` score of each term-document in the current index

In [25]:
def update_index(index, collection_size, champion_lists_ratio=0):
    
    for token, token_dict in index.items():
        document_frequency = len(token_dict['docs'])
        champion_list_size = int(champion_lists_ratio * document_frequency)
        champion_list_enable = champion_list_size > 5
        
        if champion_list_enable:
            champion_list = []
            
        for doc in token_dict['docs']:
            docID = list(doc.keys())[0]
            doc_dict = list(doc.values())[0]
            term_frequency = doc_dict['total_inc']
            tfidf_score = compute_tfidf(term_frequency, document_frequency, collection_size)
            doc_dict['tfidf'] = tfidf_score
            preprocessed_docs[docID]['length'] += tfidf_score ** 2
            
            if champion_list_enable:
                champion_list.append((doc, tfidf_score))
                
        if champion_list_enable:
            champion_list.sort(key=lambda x: x[1], reverse=True)
            token_dict['champion list'] = champion_list[:champion_list_size]

In [26]:
update_index(index, len(preprocessed_docs), champion_lists_ratio=0.5)

In [23]:
preprocessed_docs['0']['content'].count('پیام')

1

# Similarity Function

## Cosine Similarity

In [12]:
def cosine_similarity(query, K, champion_list_enable=False):
    
    scores = {}
    
    for term in query:
        
        if champion_list_enable == False:
            postings = index[term]['champion list']
            document_frequency_query = len(index[term]['docs'])
        else:
            postings = index[term]['docs']
            document_frequency_query = len(postings)
            
        term_frequency_query = query.count(term)
        
        weight_term_query = compute_tfidf(term_frequency_query, document_frequency_query, len(preprocessed_docs))
        
        for doc in postings:
            docID = list(doc.keys())[0]
            weight_term_doc = doc[docID]['tfidf']
            
            if scores.get(docID):
                scores[docID] += weight_term_query * weight_term_doc
            else:
                scores[docID] = weight_term_query * weight_term_doc
    
    docID_list = list(scores.keys())
    
    for docID in docID_list:
        
        doc_length = preprocessed_docs[docID]['length'] ** 0.5
        scores[docID] = scores[docID] / doc_length
        
    scores_tuples = list(scores.items())
    scores_tuples.sort(key=lambda x: x[1], reverse=True)
    
    return scores_tuples[:K]

## Previous Phase Query Processing

In [55]:
def previous_phase_process_query(query):
    
    intersect = set()
    
    query_index = dict()
    exception_index = dict()
    phrase_index = dict()
    
    token_docID = dict()
    exception_token_docID = dict()
    phrase_token_docID = dict()
    
    exception_tokens = list()
    phrasal_tokens = list()
    
    # find tokens immediately after exclamation's mark
    exception_tokens = re.findall(r'\!\s(\w+)', query)
    # find tokens in between double quotation
    phrasal_tokens = re.findall(r'"([^"]*)"', query)
    # remove two types of tokens extracted before from main query
    raw_tokens = re.sub(r'\!\s\w+', '', query)
    raw_tokens = re.sub(r'"[^"]*"', '', raw_tokens)
            
#     print(raw_tokens)
#     print(" ".join(exception_tokens))
#     print(" ".join(phrasal_tokens))
    
    # preprocess each type of token in query
    if len(raw_tokens):
        preprocessed_query = preprocess(raw_tokens)
        preprocessed_query = [token for token in preprocessed_query if token in dictionary]
        raw_tokens = preprocessed_query
    if len(exception_tokens) > 0:
        preprocessed_exception = preprocess(" ".join(exception_tokens))
        preprocessed_exception = [token for token in preprocessed_exception if token in dictionary]
        exception_tokens = preprocessed_exception
    if len(phrasal_tokens):
        preprocessed_phrasals = list()
        for phrase in phrasal_tokens:
            preprocessed_phrasals.append(preprocess(phrase))
    
    # separate docID related to extracted raw tokens
    if len(raw_tokens):
        for token in preprocessed_query:
            query_index[token] = index[token]
            token_docID[token] = set(map(lambda x: int(list(x.keys())[0]), index[token]['docs']))

    # separate docID related to extracted negated tokens
    if len(exception_tokens) > 0:
        for token in preprocessed_exception:
            exception_index = index[token]
            exception_token_docID[token] = set(map(lambda x: int(list(x.keys())[0]), index[token]['docs']))
            
    if len(phrasal_tokens):
        for phrase in preprocessed_phrasals:
            phrase_token_docID[tuple(phrase)] = dict()
            for token in phrase:
                phrase_index = index[token]
                phrase_token_docID[tuple(phrase)][token] = set(map(lambda x: int(list(x.keys())[0]), phrase_index['docs']))
         
#         print(phrase_token_docID)
        phrase_docID = dict()
        for phrase in preprocessed_phrasals:
            phrase_docID[tuple(phrase)] = set()
            phrase_intersect = phrase_token_docID[tuple(phrase)][phrase[0]]
            for token in phrase:
                phrase_intersect = phrase_intersect.intersection(phrase_token_docID[tuple(phrase)][token])
            
            for docID in list(phrase_intersect):
#                 print(index[phrase[0]]['docs'])
                sequential_position = None
                for i in range(len(phrase)):
                    current_token = phrase[i]
                    for doc in index[current_token]['docs']:
                        if int(list(doc.keys())[0]) == docID:
                            current_position = set(doc[f'{docID}']['position'])
                            if i == 0:
                                sequential_position = current_position
                                continue
                            current_position = set(map(lambda x : x - i, current_position))
                            sequential_position = sequential_position.intersection(current_position)
                            
                if len(sequential_position) > 0:
                    phrase_docID[tuple(phrase)].add(docID)
                    
        final_phrase_docID = phrase_docID[tuple(preprocessed_phrasals[0])]
        for phrase in preprocessed_phrasals:
            final_phrase_docID = final_phrase_docID.intersection(phrase_docID[tuple(phrase)])
                
    if len(raw_tokens) > 0:
        intersect = token_docID[preprocessed_query[0]]
        for token in preprocessed_query:
            intersect = intersect.intersection(token_docID[token])
        
    if len(phrasal_tokens) > 0:
        if len(intersect) > 0:
            intersect = intersect.intersection(final_phrase_docID)
        else: 
            intersect = final_phrase_docID
        
    if len(exception_tokens) > 0:
        for token in  preprocessed_exception:
            intersect = intersect - exception_token_docID[token]
            
#     print(f'intersect: {intersect}')
        
    related_docID = list(intersect)
#     print(related_docID)
#     print(query_index)
    related_docs = list()
    for docID in related_docID:
        total_token_inc = 0
        if len(raw_tokens) > 0:
            for token in preprocessed_query:
                for doc in query_index[token]['docs']:
                    if int(list(doc.keys())[0]) == docID:
                        total_token_inc += doc[f'{docID}']['total_inc']
        if len(phrasal_tokens) > 0:
            for phrase in preprocessed_phrasals:
                for token in phrase:
                    for doc in index[token]['docs']:
                        if int(list(doc.keys())[0]) == docID:
                            total_token_inc += doc[f'{docID}']['total_inc']
        related_docs.append((docID, total_token_inc))
        
    related_docs.sort(key=lambda x : x[1], reverse=True)
        
    return related_docs[:5]

## Interface Function to Retrieve Documents

In [27]:
def process_query(query, K=5, champion_list_enable=True, previous_phase=False):
    if previous_phase:
        results = previous_phase_process_query(query)
        results = list(map(lambda x: (str(x[0]), x[1]), results))
    else:
        results = cosine_similarity(preprocess(query), K, champion_list_enable=champion_list_enable)
    for doc in results:
        print(f"docID: {doc[0]}")
        print(f'Score: {doc[1]}')
        print(f'Title: {docs[doc[0]]["title"]}')
        print(f'URL: {docs[doc[0]]["url"]}')
        print(50*'-')

In [28]:
process_query('مردم سالاری')

docID: 10011
Score: 0.8348846434455545
Title: نایب رئیس مجلس: رئیس‌جمهور بخشنامه تعیین سقف برای به‌کارگیری افراد وزارت کشور را لغو کرد
URL: https://www.farsnews.ir/news/14000918000589/نایب-رئیس-مجلس-رئیس‌جمهور-بخشنامه-تعیین-سقف-برای-به‌کارگیری-افراد
--------------------------------------------------
docID: 9078
Score: 0.8339617071189556
Title: شهید سلیمانی تراز جدیدی از مسئولیت‌پذیری را تعریف کرد
URL: https://www.farsnews.ir/news/14001013000454/شهید-سلیمانی-تراز-جدیدی-از-مسئولیت‌پذیری-را-تعریف-کرد
--------------------------------------------------
docID: 7446
Score: 0.7893949233470711
Title: کواکبیان: برای لغو تحریم‌ها باید راستی‌آزمایی و تضمین داشته باشیم
URL: https://www.farsnews.ir/news/14001205000409/کواکبیان-برای-لغو-تحریم‌ها-باید-راستی‌آزمایی-و-تضمین-داشته-باشیم
--------------------------------------------------
docID: 3299
Score: 0.7573491262384539
Title: بگوویچ سرمربی شاهین بوشهر شد
URL: https://www.farsnews.ir/news/14001110000883/بگوویچ-سرمربی-شاهین-بوشهر-شد
------------------

# Queries

## Simple Uni-word query

In [49]:
process_query('ایران')

docID: 3514
Score: 0.11689068166453247
Title: آمار بازی ایران-عراق/یوزها زهردار از شیرهای بین النهرین
URL: https://www.farsnews.ir/news/14001107000656/آمار-بازی-ایران-عراق-یوزها-زهردار-از-شیرهای-بین-النهرین
--------------------------------------------------
docID: 4257
Score: 0.10919149136317557
Title: سرمربی استرالیا: ایران می‌تواند صدرنشین شود/ از علاقمندان به حافظ و سعدی هستم
URL: https://www.farsnews.ir/news/14001028001050/سرمربی-استرالیا-ایران-می‌تواند-صدرنشین-شود-از-علاقمندان-به-حافظ-و
--------------------------------------------------
docID: 345
Score: 0.10815923289734948
Title: مسابقات قهرمانی آسیا| پیروزی دلچسب دختران هندبالیست جوان ایران مقابل هند
URL: https://www.farsnews.ir/news/14001219000570/مسابقات-قهرمانی-آسیا|-پیروزی-دلچسب-دختران-هندبالیست-جوان-ایران-مقابل
--------------------------------------------------
docID: 9893
Score: 0.10333959012113696
Title: با حکم رئیس‌جمهور، مختارپور رئیس سازمان اسناد و کتابخانه ملی شد
URL: https://www.farsnews.ir/news/14000921000552/با-حکم

### DocID = 3514
![image.png](./images/iran-1.png)

### DocID = 4257
![image.png](./images/iran-2.png)

### DocID = 345
![image.png](./images/iran-3.png)

### DocID = 9893
![image.png](./images/iran-4.png)

### DocID = 2665
![image.png](./images/iran-5.png)

<div dir="rtl">
    همانطور که مشاهده می شود، اولویت و ارجحیت به داک های با طول کمتر داده شده است. دلیل این امر، نرمال سازی امتیاز داک ها می باشد. چرا که هر چه تعداد ترم های استفاده شده در داک کمتر باشد، نسبت امتیاز کسب شده توسط کوئری به سایز داک بیشتر خواهد بود. بنابراین بعد از نرمال سازی امتیاز بالاتری خواهد گرفت. در واقع، در این حالت داک هایی برگردانده می شود که از لحاظ سایز به کوئری نزدیک تر باشد.
</div>

## Simple multi-word query

In [51]:
process_query('قهرمانی تیم ملی ایران')

docID: 3204
Score: 0.5000880974631147
Title: قطر با غلبه بر بحرین بر بام هندبال آسیا ایستاد
URL: https://www.farsnews.ir/news/14001111000880/قطر-با-غلبه-بر-بحرین-بر-بام-هندبال-آسیا-ایستاد
--------------------------------------------------
docID: 3768
Score: 0.47908428060445374
Title: زخم‌کاری هندبال ایران به کویتی‌ها/ بلیت قهرمانی جهان رزرو شد
URL: https://www.farsnews.ir/news/14001104000462/زخم‌کاری-هندبال-ایران-به-کویتی‌ها-بلیت-قهرمانی-جهان-رزرو-شد
--------------------------------------------------
docID: 1966
Score: 0.4667436573379279
Title: مسابقات فوتسال کافا|تیم ملی زیر 19 سال ایران قهرمان شد
URL: https://www.farsnews.ir/news/14001128000882/مسابقات-فوتسال-کافا|تیم-ملی-زیر-9-سال-ایران-قهرمان-شد
--------------------------------------------------
docID: 1895
Score: 0.42339854670177685
Title: پست جدید برای پدر سرپرستی در بسکتبال ایران
URL: https://www.farsnews.ir/news/14001129000649/پست-جدید-برای-پدر-سرپرستی-در-بسکتبال-ایران
--------------------------------------------------
docID: 3

### DocID = 3204
![image.png](./images/national-football-1-1.png)
![image-2.png](./images/national-football-1-2.png)
![image-3.png](./images/national-football-1-3.png)

### DocID = 3768
![image.png](./images/national-football-2-1.png)
![image-2.png](./images/national-football-2-2.png)
![image-3.png](./images/national-football-2-3.png)

### DocID = 1966
![image.png](./images/national-football-3-1.png)
![image-2.png](./images/national-football-3-2.png)

### DocID = 1895
![image.png](./images/national-football-4-1.png)
![image-2.png](./images/national-football-4-2.png)

### DocID = 3578
![image.png](./images/national-football-5-1.png)
![image-2.png](./images/national-football-5-2.png)

## Complex uni-word query

In [52]:
process_query('استمهال')

docID: 11229
Score: 1.5604113891365365
Title: وام کشاورزان استمهال شود
URL: https://www.farsnews.ir/news/14000819000217/وام-کشاورزان-استمهال-شود
--------------------------------------------------
docID: 8920
Score: 1.2662751609198915
Title: اختصاصی| شورای نگهبان برای بررسی لایحه رتبه‌بندی معلمان از مجلس مهلت خواست/ طرح تسهیل مجوزهای کسب و کار رد شد
URL: https://www.farsnews.ir/news/14001019000650/اختصاصی|-شورای-نگهبان-برای-بررسی-لایحه-رتبه‌بندی-معلمان-از-مجلس-مهلت
--------------------------------------------------
docID: 7587
Score: 1.2128913136501058
Title: اعلام نظر شورای نگهبان درباره 16 طرح و لایحه
URL: https://www.farsnews.ir/news/14001130001158/اعلام-نظر-شورای-نگهبان-درباره-6-طرح-و-لایحه
--------------------------------------------------
docID: 11210
Score: 0.644744725016795
Title: درخواست نماینده پلدختر برای استمهال بدهی سیل‌زدگان به بانک‌ها
URL: https://www.farsnews.ir/news/14000819000403/درخواست-نماینده-پلدختر-برای-استمهال-بدهی-سیل‌زدگان-به-بانک‌ها
----------------------------

### DocID = 11229
![image.png](./images/complex-1-1.png)
![image-2.png](./images/complex-1-2.png)

### DocID = 8920
![image.png](./images/complex-2.png)

### DocID = 7587
![image.png](./images/complex-3-1.png)
![image-2.png](./images/complex-3-2.png)

### DocID = 11210
![image.png](./images/complex-4-1.png)
![image-2.png](./images/complex-4-2.png)

### DocID = 11475
![image.png](./images/complex-5.png)

<div dir="rtl">
    همانطور که مشاهده می شود همه داک های بازگشتی دارای لغت استمهال می باشد و ارتباط با این موضوع را داراست.
</div>

## Complex multi-word query

In [53]:
process_query('مناقشات سیاسی خاورمیانه')

docID: 11944
Score: 0.8607731054762514
Title: ایران اجازه تشکیل «شعبه 2 رژیم صهیونیستی» را در مرزهای شمال غرب خود نخواهد داد/ همسایگان بدانند صبر ایران اندازه‌ای دارد
URL: https://www.farsnews.ir/news/14000730000363/ایران-اجازه-تشکیل-شعبه-2-رژیم-صهیونیستی-را-در-مرزهای-شمال-غرب-خود
--------------------------------------------------
docID: 9161
Score: 0.6250097968050439
Title: سردار سلیمانی، شهید پروژه نرمالیزاسیون سیاسی است
URL: https://www.farsnews.ir/news/14001012000802/سردار-سلیمانی-شهید-پروژه-نرمالیزاسیون-سیاسی-است
--------------------------------------------------
docID: 2864
Score: 0.6174407469117398
Title: برگزاری نشست ستاد هماهنگی و برنامه‌ریزی جام جهانی فوتبال ۲۰۲۲
URL: https://www.farsnews.ir/news/14001116000591/برگزاری-نشست-ستاد-هماهنگی-و-برنامه‌ریزی-جام-جهانی-فوتبال-۲۰۲۲
--------------------------------------------------
docID: 7180
Score: 0.5953748980354714
Title: آمریکا اهداف شوم خود را با بحران‌سازی‌ها پیگیری می‌کند
URL: https://www.farsnews.ir/news/14001215000649/آمریکا-

### DocID = 11944
![image.png](./images/middleeast-1.png)

### DocID = 9161
![image.png](./images/middleeast-2-1.png)
![image-2.png](./images/middleeast-2-2.png)

### DocID = 2864
![image.png](./images/middleeast-3.png)

### DocID = 7180
![image.png](./images/middleeast-4.png)

### DocID = 7312
![image.png](./images/middleeast-5.png)

<div dir="rtl">
    در این کوئری، تنها داک اول دارای کلمه مناقشات می باشد و به دلیل خاص بودن این کلمه، از امتیاز بالایی برخوردار می باشد. بعد از این داک در داک های دیگر به دلیل وجود کلمه خاورمیانه که این کلمه نیز خاص می باشد، داک ها امتیاز بالاتری دارند.
    از لحاظ محتوایی، داک سوم، ارتباط کمتری به موضوع دارد و برخلاف کوئری اصلی که دارای محتوای سیاسی می باشد، جنس داک برگشتی از نوع داک های ورزشی می باشد.
</div>

# Previous Query Processing

## Simple multi-word query

In [64]:
process_query('قهرمانی تیم ملی ایران', previous_phase=True)

docID: 1633
Score: 135
Title: پسری با کفش‌های کتانی خاطره‌انگیز| رحیمی: غریبانه وارد ایران شدیم/ برخی‌ دنبال حرف‌های خاله‌زنکی هستند
URL: https://www.farsnews.ir/news/14001118000684/پسری-با-کفش‌های-کتانی-خاطره‌انگیز|-رحیمی-غریبانه-وارد-ایران-شدیم-
--------------------------------------------------
docID: 2641
Score: 113
Title: انتخاب گزینه‌ای جوان و درعین حال باتجربه/ مومنی‌مقدم چگونه سرمربی تیم جوانان شد؟
URL: https://www.farsnews.ir/news/14001118000663/انتخاب-گزینه‌ای-جوان-و-درعین-حال-باتجربه-مومنی‌مقدم-چگونه-سرمربی-تیم
--------------------------------------------------
docID: 2831
Score: 102
Title: افشای دلایل جدایی پروفسور از هندبال ایران| فرناندز:  نمی‌خواهم وارد جنگ شوم/ با مسیر فدراسیون و برخی تصمیم‌ها موافق نبودم
URL: https://www.farsnews.ir/news/14001116000751/افشای-دلایل-جدایی-پروفسور-از-هندبال-ایران|-فرناندز--نمی‌خواهم-وارد
--------------------------------------------------
docID: 5054
Score: 93
Title: جنگ هندبالی‌ها با حریفان تا دندان مسلح در خاک عربستان/ بهر نبردی بی‌امان 

### DocID = 1633
![image.png](./images/national-team-1.png)

### DocID = 2641
![image.png](./images/national-team-2.png)

### DocID = 2831
![image.png](./images/national-team-3.png)

### DocID = 5054
![image.png](./images/national-team-4.png)

### DocID = 163
![image.png](./images/national-team-5-1.png)
![image-2.png](./images/national-team-5-2.png)

<div dir="rtl">
    همانطور که مشاهده می شود، ارتباط محتوایی داک اول و آخر به موضوع کمتر می باشد. همچنین در این حالت، برخلاف مدل مبتنی بر tfidf لزوما داک های با سایز کوچک تر ارجح شمرده نمی شود و داک های طولانی تر نیز به عنوان پاسخ بازگردانده می شود.
</div>

## Complex multi-word query

In [66]:
process_query('مناقشات سیاسی خاورمیانه', previous_phase=True)

<div dir="rtl">
    همانگونه که مشاهده می شود، مدل قبلی پاسخگویی به کوئری توانایی پاسخ به این کوئری را دارا نبود و به همین دلیل در خروجی هیچ داکی برگردانده نشده است.
</div>